In [1]:
import pandas as pd
import numpy as np
import os
os.chdir('/Users/fabian_coll/Desktop/PyProjects/COVID-Race-Income-Analysis/data/dirty')

In [2]:

csv = pd.read_csv('county_race.csv')
df = pd.DataFrame(csv)

In [3]:
df = df.rename(
    {'Non-Hispanic White': 'White', 
    'Non-Hispanic Black': 'Black',
    'Non-Hispanic American Indian or Alaska Native': 'AmericanIndian/NativeIslander',
    'Non-Hispanic Asian': 'Asian',
    'Non-Hispanic Native Hawaiian or Other Pacific Islander': 'NativeHawaiian/PacificIslander',
    'Total deaths': 'TotalDeaths',
    'Start Date': 'StartDate',
    'End Date': 'EndDate',
    'County Name': 'County',
    'Urban Rural Code': 'RuralCode'}, 
    axis=1)
# Remove unwanted columns
df = df.drop(['StartDate', 'EndDate', 'RuralCode', 'Data as of', 'FIPS State', 'FIPS County', 'FIPS Code', 'Footnote', 'Urban Rural Description'], axis=1)

In [4]:
df['CountyID'] = df.apply(lambda row: str(row['County']+row['State']), axis = 1)
df

,State,County,Indicator,TotalDeaths,COVID-19 Deaths,White,Black,AmericanIndian/NativeIslander,Asian,NativeHawaiian/PacificIslander,Hispanic,Other,CountyID
0,AK,Anchorage Municipality,Distribution of all-cause deaths (%),2624,180,0.580,0.045,0.211,0.057,0.029,0.033,0.046,Anchorage MunicipalityAK
1,AK,Anchorage Municipality,Distribution of COVID-19 deaths (%),2624,180,0.317,NaN,0.339,0.150,0.089,NaN,NaN,Anchorage MunicipalityAK
2,AK,Anchorage Municipality,Distribution of population (%),2624,180,0.556,0.052,0.081,0.095,0.026,0.091,0.099,Anchorage MunicipalityAK
3,AL,Baldwin County,Distribution of all-cause deaths (%),2757,234,0.900,0.079,NaN,NaN,NaN,0.013,NaN,Baldwin CountyAL
4,AL,Baldwin County,Distribution of COVID-19 deaths (%),2757,234,0.859,0.115,NaN,NaN,NaN,NaN,NaN,Baldwin CountyAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2164,WY,Laramie County,Distribution of COVID-19 deaths (%),1185,116,0.750,NaN,NaN,NaN,NaN,0.198,NaN,Laramie CountyWY
2165,WY,Laramie County,Distribution of population (%),1185,116,0.785,0.022,0.006,0.012,0.001,0.149,0.024,Laramie CountyWY
2166,WY,Natrona County,Distribution of all-cause deaths (%),1244,173,0.908,0.008,0.035,NaN,NaN,0.043,NaN,Natrona CountyWY
2167,WY,Natrona County,Distribution of COVID-19 deaths (%),1244,173,0.815,NaN,0.116,NaN,NaN,0.058,NaN,Natrona CountyWY


In [5]:
df_alldeaths = df[(df['Indicator'] == 'Distribution of all-cause deaths (%)')]
df_COVIDdeaths = df[(df['Indicator'] == 'Distribution of COVID-19 deaths (%)')]
df_population = df[(df['Indicator'] == 'Distribution of population (%)')]

In [6]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [7]:
csv = pd.read_csv('cc-est2019-alldata.csv', encoding='latin-1')
countyStats = pd.DataFrame(csv)
countyStats = countyStats[(countyStats['YEAR'] == 12) & (countyStats['AGEGRP'] == 0)]
countyStats['State'] = countyStats.apply(lambda row: us_state_abbrev[row['STNAME']], axis = 1)
countyStats['CountyID'] = countyStats.apply(lambda row: str(row['CTYNAME']+row['State']), axis = 1)
countyStats = countyStats[countyStats['CountyID'].isin(df_COVIDdeaths.CountyID.unique())]


In [8]:
countyStats = countyStats[['CountyID', 'CTYNAME', 'State', 'TOT_POP', 'WA_MALE', 'WA_FEMALE', 'BA_MALE', 'BA_FEMALE', 'IA_MALE', 'IA_FEMALE', 'AA_MALE', 'AA_FEMALE', 'NA_MALE', 'NA_FEMALE', 'TOM_MALE', 'TOM_FEMALE', 'H_MALE', 'H_FEMALE']]


In [9]:
countyStats['White_Pop'] = countyStats['WA_MALE'] + countyStats['WA_FEMALE']
countyStats['Black_Pop'] = countyStats['BA_MALE'] + countyStats['BA_FEMALE']
countyStats['Asian_Pop'] = countyStats['AA_MALE'] + countyStats['AA_FEMALE']
countyStats['Hispanic_Pop'] = countyStats['H_MALE'] + countyStats['H_FEMALE']
countyStats['AmericanIndian/NativeIslander_Pop'] = countyStats['IA_MALE'] + countyStats['IA_FEMALE']
countyStats['NativeHawaiian/PacificIslander_Pop'] = countyStats['NA_MALE'] + countyStats['NA_FEMALE']
countyStats['Other_Pop'] = countyStats['TOM_MALE'] + countyStats['TOM_FEMALE']
countyStats['Total_Population'] = sum(
    [
    countyStats['White_Pop'], 
    countyStats['Black_Pop'], 
    countyStats['Asian_Pop'], 
    countyStats['Hispanic_Pop'], 
    countyStats['AmericanIndian/NativeIslander_Pop'], 
    countyStats['NativeHawaiian/PacificIslander_Pop'],
    ]
    )

In [10]:
pop_dict = {}
for index, row in countyStats.iterrows():
    pop_dict[row['CountyID']] = [
        row['Total_Population'],
        row['White_Pop'],
        row['Black_Pop'],
        row['Asian_Pop'],
        row['Hispanic_Pop'],
        row['AmericanIndian/NativeIslander_Pop'],
        row['NativeHawaiian/PacificIslander_Pop'],
        row['Other_Pop']
    ]

def get_population(row, val):
    try:
        return int(pop_dict[row['CountyID']][val])
    except KeyError:
        pass

df_COVIDdeaths['Total_Population'] = df_COVIDdeaths.apply(lambda row: get_population(row, 0), axis = 1)
df_COVIDdeaths['White_Pop'] = df_COVIDdeaths.apply(lambda row: get_population(row, 1), axis = 1)
df_COVIDdeaths['Black_Pop'] = df_COVIDdeaths.apply(lambda row: get_population(row, 2), axis = 1)
df_COVIDdeaths['Asian_Pop'] = df_COVIDdeaths.apply(lambda row: get_population(row, 3), axis = 1)
df_COVIDdeaths['Hispanic_Pop'] = df_COVIDdeaths.apply(lambda row: get_population(row, 4), axis = 1)
df_COVIDdeaths['AmericanIndian/NativeIslander_Pop'] = df_COVIDdeaths.apply(lambda row: get_population(row, 5), axis = 1)
df_COVIDdeaths['NativeHawaiian/PacificIslander_Pop'] = df_COVIDdeaths.apply(lambda row: get_population(row, 6), axis = 1)
df_COVIDdeaths['Other_Pop'] = df_COVIDdeaths.apply(lambda row: get_population(row, 7), axis = 1)

ipykernel_launcher:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:23: SettingWithCopyWarning: 
A value is trying t

In [11]:
df_COVIDdeaths

,State,County,Indicator,TotalDeaths,COVID-19 Deaths,White,Black,AmericanIndian/NativeIslander,Asian,NativeHawaiian/PacificIslander,...,Other,CountyID,Total_Population,White_Pop,Black_Pop,Asian_Pop,Hispanic_Pop,AmericanIndian/NativeIslander_Pop,NativeHawaiian/PacificIslander_Pop,Other_Pop
1,AK,Anchorage Municipality,Distribution of COVID-19 deaths (%),2624,180,0.317,NaN,0.339,0.15,0.089,...,NaN,Anchorage MunicipalityAK,290826.0,183670.0,17195.0,28751.0,26992.0,26179.0,8039.0,24166.0
4,AL,Baldwin County,Distribution of COVID-19 deaths (%),2757,234,0.859,0.115,NaN,NaN,NaN,...,NaN,Baldwin CountyAL,229601.0,195198.0,19593.0,2380.0,10534.0,1742.0,154.0,4167.0
7,AL,Calhoun County,Distribution of COVID-19 deaths (%),1972,267,0.798,0.191,NaN,NaN,NaN,...,NaN,Calhoun CountyAL,115905.0,85322.0,24213.0,1024.0,4614.0,605.0,127.0,2314.0
10,AL,Colbert County,Distribution of COVID-19 deaths (%),885,160,0.788,0.162,NaN,NaN,NaN,...,NaN,Colbert CountyAL,55984.0,44440.0,8959.0,374.0,1789.0,382.0,40.0,1046.0
13,AL,Cullman County,Distribution of COVID-19 deaths (%),1422,202,0.975,NaN,NaN,NaN,NaN,...,NaN,Cullman CountyAL,86310.0,80188.0,1156.0,607.0,3752.0,539.0,68.0,1210.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,WV,Monongalia County,Distribution of COVID-19 deaths (%),2222,284,0.975,NaN,NaN,NaN,NaN,...,NaN,Monongalia CountyWV,105549.0,95265.0,4103.0,3545.0,2352.0,218.0,66.0,2415.0
2158,WV,Ohio County,Distribution of COVID-19 deaths (%),1312,200,0.965,NaN,NaN,NaN,NaN,...,NaN,Ohio CountyWV,41010.0,38517.0,1517.0,335.0,537.0,91.0,13.0,938.0
2161,WV,Wood County,Distribution of COVID-19 deaths (%),1313,158,0.987,NaN,NaN,NaN,NaN,...,NaN,Wood CountyWV,83118.0,80332.0,1034.0,498.0,1018.0,198.0,38.0,1418.0
2164,WY,Laramie County,Distribution of COVID-19 deaths (%),1185,116,0.750,NaN,NaN,NaN,NaN,...,NaN,Laramie CountyWY,111394.0,91392.0,2510.0,1352.0,14843.0,1141.0,156.0,2949.0


In [12]:
df_population

,State,County,Indicator,TotalDeaths,COVID-19 Deaths,White,Black,AmericanIndian/NativeIslander,Asian,NativeHawaiian/PacificIslander,Hispanic,Other,CountyID
2,AK,Anchorage Municipality,Distribution of population (%),2624,180,0.556,0.052,0.081,0.095,0.026,0.091,0.099,Anchorage MunicipalityAK
5,AL,Baldwin County,Distribution of population (%),2757,234,0.832,0.086,0.007,0.011,0.001,0.047,0.017,Baldwin CountyAL
8,AL,Calhoun County,Distribution of population (%),1972,267,0.716,0.210,0.004,0.009,0.001,0.041,0.019,Calhoun CountyAL
11,AL,Colbert County,Distribution of population (%),885,160,0.777,0.160,0.006,0.007,0.001,0.032,0.018,Colbert CountyAL
14,AL,Cullman County,Distribution of population (%),1422,202,0.916,0.012,0.006,0.007,0.001,0.045,0.013,Cullman CountyAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2156,WV,Monongalia County,Distribution of population (%),2222,284,0.883,0.038,0.002,0.033,0.000,0.022,0.022,Monongalia CountyWV
2159,WV,Ohio County,Distribution of population (%),1312,200,0.919,0.035,0.002,0.008,0.000,0.013,0.022,Ohio CountyWV
2162,WV,Wood County,Distribution of population (%),1313,158,0.951,0.012,0.002,0.006,0.000,0.012,0.017,Wood CountyWV
2165,WY,Laramie County,Distribution of population (%),1185,116,0.785,0.022,0.006,0.012,0.001,0.149,0.024,Laramie CountyWY


In [22]:
def get_deathsPer100k(row, raceRow, racePopRow):
    return ((row[raceRow]*row['COVID-19 Deaths']) / row[racePopRow])*100000

df_COVIDdeaths['White_per100k'] = df_COVIDdeaths.apply(lambda row: get_deathsPer100k(row, 'White', 'White_Pop'), axis = 1)

df_COVIDdeaths['Black_per100k'] = df_COVIDdeaths.apply(lambda row: get_deathsPer100k(row, 'Black', 'Black_Pop'), axis = 1)

df_COVIDdeaths['AmericanIndian/NativeIslander_per100k'] = df_COVIDdeaths.apply(lambda row: get_deathsPer100k(row, 'AmericanIndian/NativeIslander', 'AmericanIndian/NativeIslander_Pop'), axis = 1)

df_COVIDdeaths['Asian_per100k'] = df_COVIDdeaths.apply(lambda row: get_deathsPer100k(row, 'Asian', 'Asian_Pop'), axis = 1)

df_COVIDdeaths['NativeHawaiian/PacificIslander_per100k'] = df_COVIDdeaths.apply(lambda row: get_deathsPer100k(row, 'NativeHawaiian/PacificIslander', 'NativeHawaiian/PacificIslander_Pop'), axis = 1)

df_COVIDdeaths['Hispanic_per100k'] = df_COVIDdeaths.apply(lambda row: get_deathsPer100k(row, 'Hispanic', 'Hispanic_Pop'), axis = 1)

df_COVIDdeaths['Other_per100k'] = df_COVIDdeaths.apply(lambda row: get_deathsPer100k(row, 'Other', 'Other_Pop'), axis = 1)

df_COVIDdeaths

ipykernel_launcher:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:10: SettingWithCopyWarning: 
A value is trying to b

,State,County,Indicator,TotalDeaths,COVID-19 Deaths,White,Black,AmericanIndian/NativeIslander,Asian,NativeHawaiian/PacificIslander,...,AmericanIndian/NativeIslander_Pop,NativeHawaiian/PacificIslander_Pop,Other_Pop,White_per100k,Black_per100k,AmericanIndian/NativeIslander_per100k,Asian_per100k,NativeHawaiian/PacificIslander_per100k,Hispanic_per100k,Other_per100k
1,AK,Anchorage Municipality,Distribution of COVID-19 deaths (%),2624,180,0.317,NaN,0.339,0.15,0.089,...,26179.0,8039.0,24166.0,31.066587,NaN,233.087589,93.909777,199.278517,NaN,NaN
4,AL,Baldwin County,Distribution of COVID-19 deaths (%),2757,234,0.859,0.115,NaN,NaN,NaN,...,1742.0,154.0,4167.0,102.975440,137.344970,NaN,NaN,NaN,NaN,NaN
7,AL,Calhoun County,Distribution of COVID-19 deaths (%),1972,267,0.798,0.191,NaN,NaN,NaN,...,605.0,127.0,2314.0,249.719885,210.618263,NaN,NaN,NaN,NaN,NaN
10,AL,Colbert County,Distribution of COVID-19 deaths (%),885,160,0.788,0.162,NaN,NaN,NaN,...,382.0,40.0,1046.0,283.708371,289.318004,NaN,NaN,NaN,NaN,NaN
13,AL,Cullman County,Distribution of COVID-19 deaths (%),1422,202,0.975,NaN,NaN,NaN,NaN,...,539.0,68.0,1210.0,245.610316,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,WV,Monongalia County,Distribution of COVID-19 deaths (%),2222,284,0.975,NaN,NaN,NaN,NaN,...,218.0,66.0,2415.0,290.662888,NaN,NaN,NaN,NaN,NaN,NaN
2158,WV,Ohio County,Distribution of COVID-19 deaths (%),1312,200,0.965,NaN,NaN,NaN,NaN,...,91.0,13.0,938.0,501.077446,NaN,NaN,NaN,NaN,NaN,NaN
2161,WV,Wood County,Distribution of COVID-19 deaths (%),1313,158,0.987,NaN,NaN,NaN,NaN,...,198.0,38.0,1418.0,194.126873,NaN,NaN,NaN,NaN,NaN,NaN
2164,WY,Laramie County,Distribution of COVID-19 deaths (%),1185,116,0.750,NaN,NaN,NaN,NaN,...,1141.0,156.0,2949.0,95.194328,NaN,NaN,NaN,NaN,154.739608,NaN


In [21]:
os.chdir('/Users/fabian_coll/Desktop/PyProjects/COVID-Race-Income-Analysis/data/clean')
df_COVIDdeaths.to_csv('COVID_deaths.csv', index = False)